## Data Analytics

In [328]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from pymongo import MongoClient
import json, re
from re import sub
from decimal import Decimal
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.dataframe as dd
from dask.delayed import delayed

### Dask Client

In [329]:
client = Client(n_workers=4, threads_per_worker=1)
client

/home/stivenramireza/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:40483 Dashboard: http://127.0.0.1:34315/status,Cluster Workers: 4 Cores: 4 Memory: 16.63 GB


### Database Connection

In [330]:
DB_USER = 'anutibara'
DB_PASS = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'scraping_db'

try:
    client = MongoClient(f'mongodb+srv://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except Exception as e:
    print("Error to connect to database: ", e)
db = client.get_database(DB_NAME)
properties = db.properties

Database connected successfully


### Database Queries

In [331]:
properties.count_documents({})

14209

### Convert Mongo Collection to DataFrame

#### New Properties

In [332]:
new_properties_json = list(properties.find({ 'use': 'Nuevo' }))

In [333]:
df_general_info = json_normalize(new_properties_json, record_path='offersType', 
                                meta=['_id', 
                                    'urlProperty',
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [334]:
df_location = json_normalize(new_properties_json, record_path='location', meta='urlProperty')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [335]:
df_owner_property = json_normalize(new_properties_json, record_path='ownerProperty', meta='urlProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [336]:
df_features = json_normalize(new_properties_json, record_path='features', meta='urlProperty')
ddf_features = dd.from_pandas(df_features, npartitions=10)

In [337]:
ddf_features = ddf_features.rename(columns={
    'price':'range_prices',
    'rooms': 'general_rooms',
    'bathrooms': 'general_bathrooms',
    'privateArea': 'range_private_area'
})

In [338]:
df_more_features = json_normalize(new_properties_json, record_path='moreFeatures', meta='urlProperty')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [339]:
ddf_list = [ddf_general_info, ddf_location, ddf_owner_property, ddf_features, ddf_more_features]
for ddf in ddf_list:
    ddf.set_index('urlProperty')

ddf = dd.concat(ddf_list, axis=1)
ddf.reset_index()

,index,property,offerType,area,privateArea,rooms,bathrooms,price,_id,urlProperty,scrapingDate,scrapingHour,modifyDate,modifyHour,code,status,type,use,nameProject,description,country,department,city,sector,neighborhood,address,latitude,longitude,urlProperty,id,name,contractType,financing,schedule,urlProperty,range_prices,squareMeters,general_rooms,general_bathrooms,garages,range_private_area,constructionArea,squareMetersPrice,stratum,condition,antiquity,floor,interiorFloors,weather,includesAdministration,admonPrice,urlProperty,interiorFeatures,exteriorFeatures,sectorFeatures,urlProperty
npartitions=21,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,object,int64,object,object,object,object,object,object,object,int64,int64,object,object,object,float64,object,object,object,object,int64,object,object,float64,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [340]:
ddf_new_properties = ddf.loc[:,~ddf.columns.duplicated()]
ddf_new_properties = ddf_new_properties.dropna()

#### Old Properties

In [341]:
old_properties_json = list(properties.find({ 'use': 'Usado' }))

In [342]:
df_general_info = pd.DataFrame(old_properties_json, 
                                    columns=['_id', 
                                    'urlProperty', 
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [343]:
df_location = json_normalize(old_properties_json, 'location')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [344]:
df_owner_property = json_normalize(old_properties_json, 'ownerProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [345]:
df_features = json_normalize(old_properties_json, 'features')
ddf_features = dd.from_pandas(df_features, npartitions=10)

In [346]:
df_more_features = json_normalize(old_properties_json, 'moreFeatures')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [347]:
ddf_list = [ddf_general_info, ddf_location, ddf_owner_property, ddf_features, ddf_more_features]
ddf = dd.concat(ddf_list, axis=1)
ddf.reset_index()

,index,_id,urlProperty,scrapingDate,scrapingHour,modifyDate,modifyHour,code,status,type,use,nameProject,description,country,department,city,sector,neighborhood,address,latitude,longitude,id,name,contractType,financing,schedule,price,squareMeters,rooms,bathrooms,garages,privateArea,constructionArea,squareMetersPrice,stratum,condition,antiquity,floor,interiorFloors,weather,includesAdministration,admonPrice,interiorFeatures,exteriorFeatures,sectorFeatures
npartitions=20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,int64,object,object,object,object,object,object,object,object,object,object,object,float64,float64,int64,object,object,object,object,object,object,int64,int64,object,object,object,float64,object,object,object,object,int64,object,object,float64,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [348]:
ddf_old_properties = ddf.loc[:,~ddf.columns.duplicated()]
ddf_old_properties = ddf_old_properties.dropna()

### Rename DataFrame Columns

#### New Properties

In [349]:
ddf_new_properties = ddf_new_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "id": "id_owner_property",
                        "name": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "squareMeters": "range_square_meters",
                        "constructionArea": "range_construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "interiorFeatures": "interior_features",
                        "exteriorFeatures": "exterior_features",
                        "sectorFeatures": "sector_features",
                        "offerType": "offer_type",
                        "privateArea": "private_area"
})

#### Old Properties

In [350]:
ddf_old_properties = ddf_old_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "id": "id_owner_property",
                        "name": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "squareMeters": "square_meters",
                        "privateArea": "private_area",
                        "constructionArea": "construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "interiorFeatures": "interior_features",
                        "exteriorFeatures": "exterior_features",
                        "sectorFeatures": "sector_features"
})

### Data Cleaning

#### New Properties

In [351]:
ddf_new_properties['id_mongoose'] = ddf_new_properties['id_mongoose'].astype(str)
ddf_new_properties['code'] = ddf_new_properties['code'].astype(int)
ddf_new_properties['active'] = ddf_new_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_new_properties['new_property'] = ddf_new_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_new_properties['includes_administration'] = ddf_new_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_new_properties['garages'] = ddf_new_properties['garages'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['stratum'] = ddf_new_properties['stratum'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['floor'] = ddf_new_properties['floor'].astype(int)
ddf_new_properties['area'] = ddf_new_properties['area'].astype(float)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].replace('', value = 0, regex = True)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].astype(float)
ddf_new_properties['rooms'] = ddf_new_properties['rooms'].astype(int)
ddf_new_properties['bathrooms'] = ddf_new_properties['bathrooms'].astype(int)
ddf_new_properties['price'] = ddf_new_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_new_properties['price'] = ddf_new_properties['price'].astype(int)

#### Old Properties

In [352]:
ddf_old_properties['id_mongoose'] = ddf_old_properties['id_mongoose'].astype(str)
ddf_old_properties['code'] = ddf_old_properties['code'].astype(int)
ddf_old_properties['active'] = ddf_old_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_old_properties['new_property'] = ddf_old_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_old_properties['includes_administration'] = ddf_old_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('Más de 10', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].astype(int)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('Campestre', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].astype(int)
ddf_old_properties['price'] = ddf_old_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_old_properties['price'] = ddf_old_properties['price'].astype(int)
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].str[0:-3]
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].apply(lambda meters: Decimal(sub(r'[^\d,]', '', meters)), meta=('square_meters', 'float'))
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].astype(float)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].str[0:-2]
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].replace('', value = '0', regex = True)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].apply(lambda area: Decimal(sub(r'[^\d.]', '', area)), meta=('private_area', 'float'))
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].astype(float)
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].str[0:-3]
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].apply(lambda area: Decimal(sub(r'[^\d,]', '', area)), meta=('construction_area', 'float'))
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].astype(float)
ddf_old_properties['floor'] = ddf_old_properties['floor'].astype(int)

### Data Analytics after Data Cleaning

#### New Properties

In [353]:
ddf_new_properties.describe().head(20)

,area,private_area,rooms,bathrooms,price,code,latitude,longitude,id_owner_property,general_rooms,general_bathrooms,garages,square_meters_price,stratum,floor,interior_floors,admon_price
count,4512.000000,4512.000000,4512.000000,4512.00000,4.512000e+03,4.512000e+03,4512.000000,4512.000000,4.512000e+03,4512.000000,4512.000000,4512.000000,4.512000e+03,4512.000000,4512.0,4512.0,4512.000000
mean,82.901642,56.950734,2.470523,2.32070,4.957254e+08,2.972233e+06,3.496307,-42.394599,5.627891e+05,1.922651,1.941711,0.773050,5.670606e+06,4.914894,0.0,0.0,80.895390
std,22.028637,35.957174,0.808687,0.79079,1.739462e+08,7.561409e+05,3.093565,37.510672,5.974222e+05,0.861760,0.697983,0.715537,1.485622e+06,0.951183,0.0,0.0,5433.858424
min,41.750000,0.000000,1.000000,1.00000,2.062000e+08,1.518663e+06,0.000000,-75.631317,2.848100e+04,1.000000,1.000000,0.000000,1.428571e+06,2.000000,0.0,0.0,0.000000
25%,72.460000,41.760000,2.000000,2.00000,3.855716e+08,2.565842e+06,0.000000,-75.573837,1.678440e+05,2.000000,2.000000,0.000000,4.521514e+06,4.000000,0.0,0.0,0.000000
50%,87.500000,78.080000,3.000000,3.00000,4.868707e+08,2.800362e+06,6.223497,0.000000,3.834980e+05,2.000000,2.000000,1.000000,5.077692e+06,5.000000,0.0,0.0,0.000000
75%,103.940000,91.860000,3.000000,3.00000,6.446640e+08,3.987571e+06,6.225793,0.000000,1.404447e+06,3.000000,3.000000,2.000000,7.535583e+06,6.000000,0.0,0.0,0.000000
max,134.900000,105.050000,4.000000,5.00000,9.649300e+08,4.981342e+06,6.282019,0.000000,1.804954e+06,3.000000,4.000000,2.000000,7.701619e+06,6.000000,0.0,0.0,365000.000000


#### Old Properties

In [355]:
ddf_old_properties.describe().head(20)

,code,latitude,longitude,id_owner_property,price,square_meters,rooms,bathrooms,garages,private_area,construction_area,square_meters_price,stratum,floor,interior_floors,admon_price
count,8.394000e+03,8394.000000,8394.000000,8.394000e+03,8.394000e+03,8394.000000,8394.000000,8394.000000,8394.000000,8394.000000,8394.000000,8.394000e+03,8394.000000,8394.000000,8394.000000,8.394000e+03
mean,4.685359e+06,5.212406,-63.100537,6.322943e+05,1.246295e+09,335.213248,2.945556,2.739576,1.096497,96.519635,335.213248,9.803312e+06,4.726710,3.330355,0.030617,6.531673e+05
std,5.375286e+05,2.565071,28.283032,5.526036e+05,5.028364e+10,4240.805061,1.662237,1.719545,1.171255,119.623371,4240.805061,3.368005e+08,1.303611,5.118458,0.266286,1.699565e+07
min,1.783039e+06,-22.735910,-98.206558,5.260000e+02,1.900000e+06,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.111110e+03,0.000000,0.000000,0.000000,0.000000e+00
25%,4.808873e+06,6.201599,-75.582335,3.455800e+05,3.300000e+08,83.500000,3.000000,2.000000,1.000000,0.000000,83.500000,3.229864e+06,4.000000,0.000000,0.000000,0.000000e+00
50%,4.952470e+06,6.217094,-75.566360,5.051510e+05,5.050000e+08,125.000000,3.000000,3.000000,1.000000,93.000000,125.000000,4.034364e+06,5.000000,1.000000,0.000000,1.873500e+05
75%,5.018642e+06,6.244705,0.000000,1.178183e+06,8.837500e+08,226.000000,4.000000,4.000000,2.000000,176.500000,226.000000,5.460599e+06,6.000000,7.000000,0.000000,4.275000e+05
max,5.047436e+06,65.421509,227.812500,1.833631e+06,4.600000e+12,280000.000000,47.000000,48.000000,10.000000,995.000000,280000.000000,3.000000e+10,6.000000,18.000000,5.000000,1.050000e+09
